# Motion

Luckily, the motion files are also the same across versions. It also looks like another archive with a table at the back of the file.

In [1]:
from pathlib import Path
from struct import Struct, unpack_from

ARCHIVE_FOOTER = Struct("<2I")
ARCHIVE_RECORD = Struct("<2I64s76x")


def extract_archive(data):
    offset = len(data) - ARCHIVE_FOOTER.size
    _, count = ARCHIVE_FOOTER.unpack_from(data, offset)
    for _ in range(count):
        offset -= ARCHIVE_RECORD.size  # walk the table backwards
        start, length, name = ARCHIVE_RECORD.unpack_from(data, offset)
        name = name.rstrip(b"\x00").decode("ascii")
        yield name, data[start : start + length]


data = Path("install/v1.0-us-pre/zbd/motion.zbd").read_bytes()
motions = dict(extract_archive(data))
print("\n".join(motions.keys()))

annihilator_collapser
vulture_trot
vulture_step
vulture_stand
vulture_run
vulture_limpb
vulture_limp
vulture_landf
vulture_landb
vulture_getupf
vulture_getupb
vulture_getup
vulture_fallf
vulture_fallb
vulture_collapser
thor_walk
thor_trot
thor_step
thor_stand
thor_run
thor_limpb
thor_limp
thor_landf
thor_landb
thor_getupf
thor_getupb
thor_fallf
thor_fallb
thor_collapser
supernova_walk
supernova_trot
supernova_step
supernova_stand
supernova_limpb
supernova_limp
supernova_landf
supernova_landb
supernova_getupf
supernova_getupb
supernova_fallf
supernova_fallb
supernova_collapser
sunder_walk
sunder_trot
sunder_step
sunder_stand
sunder_run
sunder_limpb
sunder_limp
sunder_landf
sunder_landb
sunder_getupf
sunder_getupb
sunder_fallf
sunder_fallb
sunder_collapser
strider_walk
strider_trot
strider_step
strider_stand
strider_run
strider_limpb
strider_limp
strider_landf
strider_landb
strider_getupf
strider_getupb
strider_fallf
strider_fallb
strider_collapser
shadowcat_walk
shadowcat_trot
shadowcat

In [2]:
by_length = sorted((len(motion), name) for name, motion in motions.items())
by_length[:4]

[(850, 'elemental_stand'),
 (850, 'sunder_stand'),
 (850, 'thor_stand'),
 (992, 'cauldron_stand')]

In [3]:
from helpers import hexdump

motion = motions["thor_stand"]
hexdump(motion)

000|04 00 00 00 89 88 08 3D|.......=
008|01 00 00 00 0C 00 00 00|........
016|00 00 80 BF 00 00 80 3F|.......?
024|03 00 00 00 68 69 70 0C|....hip.
032|00 00 00 13 9C FA B9 77|.......w
040|2E BD 40 87 F8 13 3F 13|..@...?.
048|9C FA B9 77 2E BD 40 87|...w..@.
056|F8 13 3F 00 00 80 3F 00|..?...?.
064|00 00 00 00 00 00 00 00|........
072|00 00 00 00 00 80 3F 00|......?.
080|00 00 00 00 00 00 00 00|........
088|00 00 00 05 00 00 00 74|.......t
096|6F 72 73 6F 0C 00 00 00|orso....
104|93 1A 5A B7 E0 65 66 3F|..Z..ef?
112|CF 11 59 3E 93 1A 5A B7|..Y>..Z.
120|E0 65 66 3F CF 11 59 3E|.ef?..Y>
128|00 00 80 3F 00 00 00 00|...?....
136|00 00 00 00 BD 37 86 B5|.....7..
144|00 00 80 3F 00 00 00 00|...?....
152|00 00 00 00 BD 37 86 B5|.....7..
160|06 00 00 00 6C 74 68 69|....lthi
168|67 68 0C 00 00 00 82 56|gh.....V
176|98 BF E0 A0 19 3F 30 4C|.....?0L
184|96 3E 82 56 98 BF E0 A0|.>.V....
192|19 3F 30 4C 96 3E 3E 59|.?0L.>>Y
200|7E BF 5E 3D E8 BD D9 DE|~.^=....
208|B6 B9 9C 85 4D BA 3E 59|....M.>Y
2

I see 'mech part names and the length of those names. A bit of manually reading the offsets later...

In [4]:
def name(offset):
    l, = unpack_from("<I", motion, offset)
    offset += 4
    print(motion[offset : offset + l])
    offset += l
    print(unpack_from("<I", motion, offset))
    return l


l = name(24)
print(91 - 24 - l)
l = name(91)
print(160 - 91 - l)
name(160)
name(230)
name(300)
name(369)
name(438)
name(507)
name(576)
name(644)
name(712)
l = name(781)
print(781 - 712 - l)

b'hip'
(12,)
64
b'torso'
(12,)
64
b'lthigh'
(12,)
b'rthigh'
(12,)
b'rfoot'
(12,)
b'lfoot'
(12,)
b'rcalf'
(12,)
b'lcalf'
(12,)
b'larm'
(12,)
b'rarm'
(12,)
b'rhand'
(12,)
b'lhand'
(12,)
64


Seems like the motion data for each armature is 64 (or 60, disregarding the length). And there are 12 armatures:

In [5]:
unpack_from("<I4x2I8x", motion, 0)

(4, 1, 12)

In [6]:
def parse_motion(name):
    motion = motions[name]
    unk1, unk2, count = unpack_from("<I4x2I8x", motion, 0)
    print(name, unk1, unk2, count)
    offset = 24
    for _ in range(count):
        l, = unpack_from("<I", motion, offset)
        offset += 4
        part = motion[offset : offset + l].decode("ascii")
        print(part, end=" ")
        offset += l
        val, = unpack_from("<I", motion, offset)
        print(val, end=" ")
        offset += 60
    print()


parse_motion("thor_stand")
parse_motion("elemental_stand")
parse_motion("sunder_stand")
parse_motion("cauldron_stand")

thor_stand 4 1 12
hip 12 torso 12 lthigh 12 rthigh 12 rfoot 12 lfoot 12 rcalf 12 lcalf 12 larm 12 rarm 12 rhand 12 lhand 12 
elemental_stand 4 1 12
hip 12 torso 12 lthigh 12 rthigh 12 rfoot 12 lfoot 12 rcalf 12 lcalf 12 larm 12 rarm 12 rhand 12 lhand 12 
sunder_stand 4 1 12
hip 12 torso 12 lthigh 12 rthigh 12 rfoot 12 lfoot 12 rcalf 12 lcalf 12 larm 12 rarm 12 rhand 12 lhand 12 
cauldron_stand 4 1 14
hip 12 torso 12 lthigh 12 rthigh 12 rfoot 12 lfoot 12 ltoe02 12 ltoe01 12 rtoe02 12 rtoe01 12 rcalf 12 lcalf 12 larm 12 rarm 12 


Looking good so far. But it does break for some of the other ones. Presumably, there's more information in the header I need to understand.

In [7]:
for name, motion in motions.items():
    four, unk2, count = unpack_from("<I4x2I4x", motion, 0)
    assert four == 4
    print(name, unk2, count)

annihilator_collapser 60 18
vulture_trot 44 6
vulture_step 61 22
vulture_stand 99 22
vulture_run 45 22
vulture_limpb 43 22
vulture_limp 40 22
vulture_landf 21 20
vulture_landb 19 18
vulture_getupf 76 20
vulture_getupb 87 18
vulture_getup 52 22
vulture_fallf 24 20
vulture_fallb 19 18
vulture_collapser 66 22
thor_walk 56 12
thor_trot 47 12
thor_step 59 12
thor_stand 1 12
thor_run 48 22
thor_limpb 43 22
thor_limp 32 22
thor_landf 18 12
thor_landb 18 12
thor_getupf 85 12
thor_getupb 79 12
thor_fallf 22 12
thor_fallb 22 12
thor_collapser 60 22
supernova_walk 56 18
supernova_trot 47 18
supernova_step 59 18
supernova_stand 1 18
supernova_limpb 44 18
supernova_limp 32 18
supernova_landf 20 18
supernova_landb 20 18
supernova_getupf 76 18
supernova_getupb 86 18
supernova_fallf 25 18
supernova_fallb 19 18
supernova_collapser 56 18
sunder_walk 56 12
sunder_trot 47 12
sunder_step 59 12
sunder_stand 1 12
sunder_run 48 14
sunder_limpb 43 22
sunder_limp 32 22
sunder_landf 18 12
sunder_landb 18 12
sund

`strider_stand` has an unknown value of 2, which should make investigation easier.

In [8]:
motion = motions["strider_stand"]
count = 22
hexdump(motion)

0000|04 00 00 00 89 88 88 3D|.......=
0008|02 00 00 00 16 00 00 00|........
0016|00 00 80 BF 00 00 80 3F|.......?
0024|03 00 00 00 68 69 70 0C|....hip.
0032|00 00 00 BD 37 86 B5 CF|....7...
0040|32 BB 40 16 C3 F5 BD BD|2.@.....
0048|37 86 B5 CF 32 BB 40 16|7...2.@.
0056|C3 F5 BD BD 37 86 B5 CF|....7...
0064|32 BB 40 16 C3 F5 BD 00|2.@.....
0072|00 80 3F 00 00 00 00 00|..?.....
0080|00 00 00 AC C5 A7 36 00|......6.
0088|00 80 3F 00 00 00 00 00|..?.....
0096|00 00 00 AC C5 A7 36 00|......6.
0104|00 80 3F 00 00 00 00 00|..?.....
0112|00 00 00 AC C5 A7 36 05|......6.
0120|00 00 00 74 6F 72 73 6F|...torso
0128|0C 00 00 00 CF 30 B5 3D|.....0.=
0136|33 4E A1 3F 01 A2 F0 BE|3N.?....
0144|CF 30 B5 3D 33 4E A1 3F|.0.=3N.?
0152|01 A2 F0 BE CF 30 B5 3D|.....0.=
0160|33 4E A1 3F 01 A2 F0 BE|3N.?....
0168|00 00 80 3F F2 37 06 B5|...?.7..
0176|88 37 06 B5 9D 53 C9 B6|.7...S..
0184|00 00 80 3F F6 37 06 B5|...?.7..
0192|84 37 06 B5 94 1A DA B6|.7......
0200|00 00 80 3F F2 37 06 B5|...?.7..
0208|88 37 0

In [9]:
print(motion[28 : 28 + 3])
print(motion[123 : 123 + 5])
print(123 - 28 - 4 - 3)
print(motion[220 : 220 + 6])
print(220 - 123 - 4 - 5)
print(motion[318 : 318 + 6])
print(318 - 220 - 4 - 6)
print(motion[416 : 416 + 5])
print(416 - 318 - 4 - 6)

b'hip'
b'torso'
88
b'lthigh'
88
b'rthigh'
88
b'rfoot'
88


I'm suspecting that number is the number of frames in the animation. So I'll try `orion_landf` or `orion_landb` next, with 18.

In [10]:
motion = motions["orion_landf"]
count = 16
hexdump(motion)

0000|04 00 00 00 9A 99 19 3F|.......?
0008|12 00 00 00 10 00 00 00|........
0016|00 00 80 BF 00 00 80 3F|.......?
0024|03 00 00 00 68 69 70 0C|....hip.
0032|00 00 00 EA AF 57 BD 4B|.....W.K
0040|8F EC 3F 74 34 AE C0 DC|..?t4...
0048|66 6A BD 9C 8B B5 3F 82|fj....?.
0056|1D AF C0 79 B1 70 BD EA|...y.p..
0064|AF 85 3F EF F2 AE C0 47|..?....G
0072|3E 6F BD CC B4 91 3F 86|>o....?.
0080|5A AF C0 E5 9B 6D BD C8|Z....m..
0088|98 AF 3F 63 C4 AF C0 59|..?c...Y
0096|E0 6B BD E3 FA D1 3F 6E|.k....?n
0104|30 B0 C0 A7 21 6A BD 03|0...!j..
0112|7A EB 3F 86 9E B0 C0 97|z.?.....
0120|70 68 BD 69 52 EE 3F 95|ph.iR.?.
0128|0E B1 C0 3B E4 66 BD 0A|...;.f..
0136|4C D3 3F 7E 80 B1 C0 3F|L.?~...?
0144|8B 65 BD 9F 22 AB 3F 28|.e..".?(
0152|F4 B1 C0 60 75 64 BD A4|...`ud..
0160|FF 89 3F 75 69 B2 C0 70|..?ui..p
0168|B4 63 BD 5D 17 82 3F 4E|.c.]..?N
0176|E0 B2 C0 F8 53 63 BD DD|....Sc..
0184|0A 8F 3F 99 58 B3 C0 E4|..?.X...
0192|67 63 BD FC 17 9E 3F 3B|gc....?;
0200|D2 B3 C0 BD FB 63 BD 54|.....c.T
0208|E0 9C 3

4480|D0 0B 57 BF D5 20 00 BF|..W.....
4488|D5 05 A4 BE D0 0B 57 BF|......W.
4496|D5 20 00 BF D5 05 A4 BE|........
4504|D0 0B 57 BF D5 20 00 BF|..W.....
4512|D5 05 A4 BE D0 0B 57 BF|......W.
4520|D5 20 00 BF D5 05 A4 BE|........
4528|D0 0B 57 BF D5 20 00 BF|..W.....
4536|D5 05 A4 BE D0 0B 57 BF|......W.
4544|D5 20 00 BF D5 05 A4 BE|........
4552|D0 0B 57 BF D5 20 00 BF|..W.....
4560|D5 05 A4 BE D0 0B 57 BF|......W.
4568|D5 20 00 BF D5 05 A4 BE|........
4576|D0 0B 57 BF D5 20 00 BF|..W.....
4584|D5 05 A4 BE D0 0B 57 BF|......W.
4592|D5 20 00 BF D5 05 A4 BE|........
4600|D0 0B 57 BF D5 20 00 BF|..W.....
4608|D5 05 A4 BE D0 0B 57 BF|......W.
4616|D5 20 00 BF D5 05 A4 BE|........
4624|D0 0B 57 BF 23 24 78 3F|..W.#$x?
4632|D5 C6 7B BE 1B 16 1B B7|..{.....
4640|FB BE 05 37 86 1B 78 3F|...7..x?
4648|84 4E 7C BE 63 28 1B B7|.N|.c(..
4656|C7 A9 05 37 0D 19 78 3F|...7..x?
4664|62 75 7C BE 9E 2D 1B B7|bu|..-..
4672|B2 A3 05 37 71 1C 78 3F|...7q.x?
4680|08 40 7C BE 6E 26 1B B7|.@|.n&..
4688|0A AC 0

In [11]:
print(motion[28 : 28 + 3])
print(motion[571 : 571 + 5])
print(571 - 28 - 4 - 3)

b'hip'
b'torso'
536


If 60 is 1, and 88 is 2, then the data per frame is 28, and the base data is 32. Sure enough, 32 + 28 * 18 = 536.

In [12]:
def parse_motion(name):
    motion = motions[name]
    four, frame_count, part_count = unpack_from("<I4x2I8x", motion, 0)
    assert four == 4
    print(name, frame_count)
    offset = 24
    frame_size = 32 + 28 * frame_count
    for _ in range(part_count):
        l, = unpack_from("<I", motion, offset)
        offset += 4
        part = motion[offset : offset + l].decode("ascii")
        print(part, end=" ")
        offset += l + frame_size
    print()


parse_motion("thor_stand")
parse_motion("orion_landf")
_, longest_name = by_length[-1]
parse_motion(longest_name)

thor_stand 1
hip torso lthigh rthigh rfoot lfoot rcalf lcalf larm rarm rhand lhand 
orion_landf 18
hip torso lthigh rthigh rfoot lfoot ltoe02 ltoe01 rtoe02 rtoe01 rcalf lcalf larm rarm rhand lhand 
vulture_stand 99
hip torso lthigh rthigh rfoot lfoot ltoe02 ltoe01 ltoe03 ltoe04 rtoe03 rtoe02 rtoe01 rtoe04 rcalf rankle lcalf lankle larm rarm rhand lhand 


28 bytes of data isn't much. But then again, from decoding 3D model, translation and rotation matrices were used. A float is 32 bits or 4 bytes, and 6 * 4 = 24.

In [13]:
def parse_motion(name):
    motion = motions[name]
    four, frame_count, part_count = unpack_from("<I4x2I8x", motion, 0)
    assert four == 4
    offset = 24
    frame_size = 28 * frame_count
    parts = {}
    for _ in range(part_count):
        l, = unpack_from("<I", motion, offset)
        offset += 4
        part_name = motion[offset : offset + l].decode("ascii")
        offset += l
        header = motion[offset : offset + 32]
        offset += 32
        frames = []
        for _ in range(frame_count):
            frame = motion[offset : offset + 28]
            offset += 28
            frames.append(frame)
        parts[part_name] = (header, frames)
    return parts, frame_count


parts, _ = parse_motion("thor_stand")
for part_name, (part_header, part_frames) in parts.items():
    print(part_name)
    print(*["{:+.2f}".format(v) for v in unpack_from("<7f", part_header, 4)])
    for part_frame in part_frames:
        print(*["{:+.2f}".format(v) for v in unpack_from("<7f", part_frame, 0)])

hip
-0.00 +5.91 +0.58 -0.00 +5.91 +0.58 +1.00
+0.00 +0.00 +0.00 +1.00 +0.00 +0.00 +0.00
torso
-0.00 +0.90 +0.21 -0.00 +0.90 +0.21 +1.00
+0.00 +0.00 -0.00 +1.00 +0.00 +0.00 -0.00
lthigh
-1.19 +0.60 +0.29 -1.19 +0.60 +0.29 -0.99
-0.11 -0.00 -0.00 -0.99 -0.11 -0.00 -0.00
rthigh
+1.19 +0.60 +0.29 +1.19 +0.60 +0.29 -1.00
-0.05 +0.00 +0.00 -1.00 -0.05 +0.00 +0.00
rfoot
+1.69 -4.53 +0.42 +1.69 -4.53 +0.42 +1.00
-0.00 +0.00 -0.00 +1.00 -0.00 +0.00 -0.00
lfoot
-1.69 -4.52 -0.29 -1.69 -4.52 -0.29 +1.00
+0.00 +0.00 +0.00 +1.00 +0.00 +0.00 +0.00
rcalf
+0.51 -3.51 +0.02 +0.51 -3.51 +0.02 -0.98
+0.19 -0.00 -0.00 -0.98 +0.19 -0.00 -0.00
lcalf
-0.50 -3.51 +0.01 -0.50 -3.51 +0.01 -0.98
+0.17 -0.00 -0.00 -0.98 +0.17 -0.00 -0.00
larm
-2.00 +2.30 +0.25 -2.00 +2.30 +0.25 +1.00
+0.00 +0.00 +0.00 +1.00 +0.00 +0.00 +0.00
rarm
+1.79 +2.32 +0.11 +1.79 +2.32 +0.11 +1.00
+0.00 +0.00 +0.00 +1.00 +0.00 +0.00 +0.00
rhand
+1.01 -0.76 +0.00 +1.01 -0.76 +0.00 +1.00
+0.00 +0.00 +0.00 +1.00 +0.00 +0.00 +0.00
lhand
-0.71 

In [14]:
parts, frame_count = parse_motion("thor_walk")
for part_name, (part_header, part_frames) in parts.items():
    i, = unpack_from("<I", part_header, 0)
    print(part_name, i, frame_count)
    print(*["{:+.2f}".format(v) for v in unpack_from("<7f", part_header, 4)])
    for part_frame in part_frames:
        print(*["{:+.2f}".format(v) for v in unpack_from("<7f", part_frame, 0)])

hip 12 56
+0.09 +5.73 -3.28 +0.11 +5.72 -3.42 +0.12
+5.72 -3.56 +0.13 +5.73 -3.71 +0.15 +5.73
-3.85 +0.16 +5.74 -3.99 +0.17 +5.76 -4.13
+0.18 +5.78 -4.27 +0.19 +5.80 -4.42 +0.19
+5.82 -4.56 +0.20 +5.84 -4.70 +0.20 +5.86
-4.84 +0.20 +5.87 -4.98 +0.19 +5.88 -5.13
+0.18 +5.88 -5.27 +0.17 +5.88 -5.41 +0.16
+5.87 -5.55 +0.14 +5.86 -5.70 +0.12 +5.85
-5.84 +0.10 +5.84 -5.98 +0.08 +5.82 -6.13
+0.05 +5.81 -6.27 +0.03 +5.79 -6.41 +0.00
+5.78 -6.56 -0.02 +5.76 -6.70 -0.04 +5.75
-6.84 -0.06 +5.74 -6.99 -0.08 +5.73 -7.13
-0.09 +5.72 -7.27 -0.11 +5.72 -7.42 -0.12
+5.73 -7.56 -0.13 +5.73 -7.70 -0.15 +5.75
-7.84 -0.16 +5.76 -7.98 -0.17 +5.78 -8.13
-0.18 +5.80 -8.27 -0.19 +5.82 -8.41 -0.20
+5.84 -8.55 -0.20 +5.86 -8.69 -0.21 +5.87
-8.84 -0.21 +5.88 -8.98 -0.20 +5.88 -9.12
-0.19 +5.88 -9.26 -0.18 +5.87 -9.41 -0.17
+5.87 -9.55 -0.15 +5.86 -9.69 -0.13 +5.84
-9.83 -0.11 +5.83 -9.98 -0.09 +5.82 -10.12
-0.06 +5.80 -10.26 -0.04 +5.79 -10.41 -0.01
+5.77 -10.55 +0.01 +5.76 -10.69 +0.03 +5.75
-10.84 +0.05 +5.74 

+1.70 -3.86 +1.35 +1.65 -3.77 +1.18 +1.61
-3.68 +1.01 +1.58 -3.59 +0.83 +1.55 -3.50
+0.65 +1.53 -3.43 +0.47 +1.51 -3.36 +0.30
+1.50 -3.31 +0.14 +1.50 -3.28 -0.02 +1.49
-3.25 -0.21 +1.50 -3.24 -0.42 +1.51 -3.24
-0.64 +1.52 -3.25 -0.87 +1.54 -3.27 -1.10
+1.56 -3.30 -1.31 +1.58 -3.35 -1.51 +1.61
-3.41 -1.68 +1.64 -3.47 -1.82 +1.66 -3.65
-1.87 +1.66 -3.94 -1.79 +1.65 -4.20 -1.64
+1.67 -4.27 -1.47 +1.00 +0.05 -0.08 +0.00
+1.00 +0.02 -0.08 +0.00 +1.00 +0.00 -0.08
+0.00 +1.00 +0.00 -0.07 +0.01 +1.00 +0.00
-0.07 +0.01 +1.00 +0.00 -0.07 +0.01 +1.00
+0.00 -0.06 +0.01 +1.00 +0.00 -0.06 +0.01
+1.00 +0.00 -0.05 +0.02 +1.00 +0.00 -0.04
+0.02 +1.00 +0.00 -0.03 +0.02 +1.00 +0.00
-0.03 +0.02 +1.00 +0.00 -0.02 +0.02 +1.00
+0.00 -0.01 +0.02 +1.00 +0.00 +0.00 +0.02
+1.00 +0.00 +0.01 +0.02 +1.00 -0.00 +0.02
+0.02 +1.00 -0.00 +0.03 +0.02 +1.00 -0.00
+0.03 +0.02 +1.00 -0.00 +0.04 +0.02 +1.00
-0.00 +0.05 +0.02 +1.00 -0.00 +0.06 +0.01
+1.00 -0.00 +0.06 +0.01 +1.00 -0.00 +0.07
+0.01 +1.00 -0.00 +0.07 +0.01 +1.0

+0.00 +0.00 +1.00 +0.00 +0.00 +0.00 +1.00
+0.00 +0.00 +0.00 +1.00 +0.00 +0.00 +0.00
+1.00 +0.00 +0.00 +0.00 +1.00 +0.00 +0.00
+0.00 +1.00 +0.00 +0.00 +0.00 +1.00 +0.00
+0.00 +0.00 +1.00 +0.00 +0.00 +0.00 +1.00
+0.00 +0.00 +0.00 +1.00 +0.00 +0.00 +0.00
+1.00 +0.00 +0.00 +0.00 +1.00 +0.00 +0.00
+0.00 +1.00 +0.00 +0.00 +0.00 +1.00 +0.00
+0.00 +0.00 +1.00 +0.00 +0.00 +0.00 +1.00
+0.00 +0.00 +0.00 +1.00 +0.00 +0.00 +0.00
+1.00 +0.00 +0.00 +0.00 +1.00 +0.00 +0.00
+0.00 +1.00 +0.00 +0.00 +0.00 +1.00 +0.00
+0.00 +0.00 +1.00 +0.00 +0.00 +0.00 +1.00
+0.00 +0.00 +0.00 +1.00 +0.00 +0.00 +0.00
+1.00 +0.00 +0.00 +0.00 +1.00 +0.00 +0.00
+0.00 +1.00 +0.00 +0.00 +0.00 +1.00 +0.00
+0.00 +0.00 +1.00 +0.00 +0.00 +0.00 +1.00
+0.00 +0.00 +0.00 +1.00 +0.00 +0.00 +0.00
rarm 12 56
+1.79 +2.32 +0.11 +1.79 +2.32 +0.11 +1.79
+2.32 +0.11 +1.79 +2.32 +0.11 +1.79 +2.32
+0.11 +1.79 +2.32 +0.11 +1.79 +2.32 +0.11
+1.79 +2.32 +0.11 +1.79 +2.32 +0.11 +1.79
+2.32 +0.11 +1.79 +2.32 +0.11 +1.79 +2.32
+0.11 +1.79 +2.32 +0.11

Hmm, I think I can see a cyclical pattern of 3 values. It's too consistent to be a coincidence. In fact, there's a pattern: 57 rows of triplets, and then 57 rows of quadruplets:

```
+0.09 +5.73 -3.28
+0.11 +5.72 -3.42
+0.12 +5.72 -3.56
[...]
+0.05 +5.74 -10.98
+0.07 +5.74 -11.12
+0.09 +5.73 -3.28
```

```
+1.00 +0.00 +0.08 +0.00
+1.00 -0.00 +0.08 -0.00
+1.00 -0.00 +0.08 -0.00
[...]
+1.00 +0.00 +0.07 +0.01
+1.00 +0.00 +0.08 +0.01
+1.00 +0.00 +0.08 +0.00
```

I'm guessing the triplets are location data, and the quadruplets are rotation. For one, because I dumped the location data when exporting the models, and it's somewhat close.

The rotation data took way longer to figure out, over a month. In the models, rotation is encoded as three rotational angles, aka. Euler angles. However, here, through much trial and error, I discovered that the rotation is encoded using [quaternions](https://en.wikipedia.org/wiki/Quaternion). Quaternions are a great way of encoding rotation that doesn't suffer from [gimbal lock](https://en.wikipedia.org/wiki/Gimbal_lock), and computationally cheap compared to matrices. But it was still a surprise to see them used here.

Even with this revelation, there are still issues. For the Mad Cat, the arms are displaced from the torso. For the Vulture, it's quite "exploded". For the Orion, the toes are displaced.

## Next up

[`Mech3Msg.dll` message table/translations extraction](09-mech3msg.ipynb)

**WARNING**: Work in progress below

In [19]:
def parse_motion(motion):
    four, unk, frame_count, part_count, minus_one, plus_one = unpack_from("<If2I2f", motion, 0)
    assert four == 4
    assert minus_one == -1.0
    assert plus_one == 1.0
    assert unk > 0.0
    offset = 24
    frame_count += 1
    parts = {}
    for _ in range(part_count):
        # name size, name, twelve?
        name_size, = unpack_from("<I", motion, offset)
        offset += 4
        part_name = motion[offset : offset + name_size].decode("ascii")
        offset += name_size
        twelve, = unpack_from("<I", motion, offset)
        assert twelve == 12
        offset += 4
        # location
        location_count = frame_count * 3
        location_data = unpack_from(f"<{location_count}f", motion, offset)
        location_values = [location_data[i:i+3] for i in range(0, location_count, 3)]
        offset += location_count * 4
        # rotation
        rotation_count = frame_count * 4
        rotation_data = unpack_from(f"<{rotation_count}f", motion, offset)
        rotation_values = [rotation_data[i:i+4] for i in range(0, rotation_count, 4)]
        offset += rotation_count * 4
        parts[part_name] = list(zip(location_values, rotation_values))
    return parts

parsed = {
    name: parse_motion(motion)
    for name, motion in motions.items()
}